In [1]:
import numpy as np
import pandas as pd

In [7]:
def gen_corners(big_pix, clusters_df, planck_nside=2048, size=64):
    import numpy as np
    import pandas as pd
    from DS_healpix_fragmentation import radec2pix, one_pixel_fragmentation, matr2dict
    
    matr = one_pixel_fragmentation(2, big_pix, 10)
    mdict = matr2dict(matr)
    clusters_df['pix'] = radec2pix(clusters_df['RA'], clusters_df['DEC'], planck_nside)
    
    corners = set()
    for pix in clusters_df['pix']:
        if pix in mdict:
            x_cl, y_cl = mdict[pix]
            cur_corners = np.array([(x, y) for x in range(max(x_cl-size+1, 0), 
                                                 min(x_cl+size-1, matr.shape[0]))
                                  for y in range(max(y_cl-size+1, 0),
                                                 min(y_cl+size-1, matr.shape[1]))])
            cur_corners = matr[cur_corners[:,0], cur_corners[:,1]]
            corners += set(cur_corners)
    
    return list(corners)

In [ ]:
def gen_data_while_train(clusters_files, big_pixels, batch_size, planck_nside=2048,
                        big_nside=2, size=64):
    import numpy as np
    import pandas as pd
    from DS_healpix_fragmentation import one_pixel_fragmentation, pix2pix, matr2dict
    
    clusters_df = pd.concat([pd.read_csv(file) for file in clusters_files], 
                            ignore_index=True)
    corners = []
    for pix in big_pixels:
        corners.extend(gen_corners(pix, clusters_df))

    big_matrs = {}
    big_mdicts = {}
    for pix in big_pixels:
        big_matrs[pix] = one_pixel_fragmentation(2, pix, 10)
        big_mdicts[pix] = matr2dict(big_matrs[pix])
    
    while 4:
        cur_corners = np.random.choice(corners, batch_size)
        big_pix = pix2pix(cur_corners, planck_nside, big_nside)
        
        small_matrs = []
        for corner, pix in zip(cur_corners, big_pix):
            x, y = big_mdicts[pix][corner]
            small_matrs.append(big_matrs[pix][x:x+size, y:y+size])
        
        